## `nmtools` Examples

This notebook shows you various array manipulation functions within `nmtools`, such as:
- [broadcasting](#Broadcasting)
- [transpose](#Transposing-an-Array)
- [reshape](#Reshaping-an-Array)
- [flattening](#Flattening-an-Array)
- [tiling](#Tiling-an-Array)
- [squeeze and unsqueeze](#Squeeze-and-Expand-Dims)

Most of the functions are similar to corresponding numpy functions. For numpy users, these functions should seems familiar and intuitive.

This notebook uses `xeus-cling` kernel to be able to use C++17 on notebook. For more info about `xeus-cling` please refer to https://github.com/jupyter-xeus/xeus-cling.

`nmtools` is an header-only library, to use it you only need to include necessary header. To add include path in `xeus-cling` simply add `pragma` for `cling`, for example:
```C++
#pragma cling add_include_path("/workspace/include")
```

In [1]:
#pragma cling add_include_path("/workspace/include")

In [2]:
#include "nmtools/nmtools.hpp"

#include <iostream>
#include <array>
#include <vector>

In file included from input_line_8:1:
In file included from /workspace/include/nmtools/nmtools.hpp:4:
/workspace/include/nmtools/constants.hpp:168:20: warning: string literal operator templates are a GNU extension [-Wgnu-string-literal-operator-template]
    constexpr auto operator ""_ct()
                   ^


In [3]:
#define DESC(x) std::cout << x << std::endl;
#define PRINT(x) \
std::cout << #x << ":\n" \
    << nm::utils::to_string(x) \
    << std::endl;

namespace nm = nmtools;
namespace na = nm::array;
namespace view = nm::view;

### Broadcasting

nmtools has broadcating rules similar to numpy. A non-owning, non-copy, "view" can also be created. Any changes on the source array reflected to the "view" object.

Some of broadcasting functions are:
- `broadcast_to`
- `broadcast_arrays`

#### broadcast_to

Broadcast single array to a given shape.

In [4]:
auto shape = std::array{3,2,3};

In [5]:
DESC("broadcasting an std::array")
{
    auto array = std::array{1,2,3};
    // non-owning non-copy "broadcast" view to array given shape
    auto broadcasted = view::broadcast_to(array,shape);
    PRINT(array)
    PRINT(broadcasted)
    PRINT(nm::shape(array))
    PRINT(nm::shape(broadcasted))
    // modify array
    array[1] = 10;
    PRINT(array)
    // changes reflected to broadcasted
    PRINT(broadcasted)
}

broadcasting an std::array
array:
[	1,	2,	3]
broadcasted:
[[[	1,	2,	3],
[	1,	2,	3]],

[[	1,	2,	3],
[	1,	2,	3]],

[[	1,	2,	3],
[	1,	2,	3]]]
nm::shape(array):
[	3]
nm::shape(broadcasted):
[	3,	2,	3]
array:
[	1,	10,	3]
broadcasted:
[[[	1,	10,	3],
[	1,	10,	3]],

[[	1,	10,	3],
[	1,	10,	3]],

[[	1,	10,	3],
[	1,	10,	3]]]


#### Broadcast Arrays

Broadcast multiple arrays together according to numpy broadcasting rules.

In [6]:
DESC("broadcasting arrays")
{
    auto array1 = std::array{1,2,3};
    int array2[2][1][1] = {
        {{4}},
        {{5}},
    };
    auto array3 = na::fixed_ndarray{{
        { 6,7, 8},
        {9,10,11},
    }};
    // non-owning non-copy "broadcast" view to array given shape
    auto [b1, b2, b3] = nm::unwrap(view::broadcast_arrays(array1,array2,array3));
    PRINT(array1)
    PRINT(array2)
    PRINT(array3)
    PRINT(b1)
    PRINT(b2)
    PRINT(b3)
    PRINT(nm::shape(b1))
    PRINT(nm::shape(b2))
    PRINT(nm::shape(b3))
}

broadcasting arrays
array1:
[	1,	2,	3]
array2:
[[[	4]],

[[	5]]]
array3:
[[	6,	7,	8],
[	9,	10,	11]]
b1:
[[[	1,	2,	3],
[	1,	2,	3]],

[[	1,	2,	3],
[	1,	2,	3]]]
b2:
[[[	4,	4,	4],
[	4,	4,	4]],

[[	5,	5,	5],
[	5,	5,	5]]]
b3:
[[[	6,	7,	8],
[	9,	10,	11]],

[[	6,	7,	8],
[	9,	10,	11]]]
nm::shape(b1):
(2,	2,	3)
nm::shape(b2):
(2,	2,	3)
nm::shape(b3):
(2,	2,	3)


In [7]:
DESC("broadcasting an std::vector")
{
    auto array = std::vector{1,2,3};
    auto broadcasted = view::broadcast_to(array,shape);
    PRINT(array)
    PRINT(broadcasted)
}

broadcasting an std::vector
array:
[	1,	2,	3]
broadcasted:
[[[	1,	2,	3],
[	1,	2,	3]],

[[	1,	2,	3],
[	1,	2,	3]],

[[	1,	2,	3],
[	1,	2,	3]]]


In [8]:
DESC("\nbroadcasting an na::fixed_ndarray")
{
    auto array = na::fixed_ndarray{{1,2,3}};
    auto broadcasted = view::broadcast_to(array,shape);
    PRINT(array)
    PRINT(broadcasted)
}


broadcasting an na::fixed_ndarray
array:
[	1,	2,	3]
broadcasted:
[[[	1,	2,	3],
[	1,	2,	3]],

[[	1,	2,	3],
[	1,	2,	3]],

[[	1,	2,	3],
[	1,	2,	3]]]


### Transposing an Array

Transpose an array given axes. Axes can be `nmtools::None`, which is a special constant. When axes is `None` reverse the order of axis.

In [9]:
auto axes = std::array{2,0,1};

In [10]:
DESC("\ntransposing an na::fixed_ndarray")
{
    auto array = na::fixed_ndarray{{
        {
            {1,2,3},
            {3,4,5}
        },
        {
            {5,6,7},
            {7,8,9}
        },
    }};
    auto transposed = view::transpose(array,axes);
    PRINT(array)
    PRINT(transposed)
    PRINT(nm::shape(array))
    PRINT(nm::shape(transposed))
}


transposing an na::fixed_ndarray
array:
[[[	1,	2,	3],
[	3,	4,	5]],

[[	5,	6,	7],
[	7,	8,	9]]]
transposed:
[[[	1,	3],
[	5,	7]],

[[	2,	4],
[	6,	8]],

[[	3,	5],
[	7,	9]]]
nm::shape(array):
[	2,	2,	3]
nm::shape(transposed):
[	3,	2,	2]


example using `None`:

In [11]:
DESC("\ntransposing a raw array")
{
    int array[2][2][3] = {
        {
            {1,2,3},
            {3,4,5}
        },
        {
            {5,6,7},
            {7,8,9}
        },
    };
    auto transposed = view::transpose(array,nm::None);
    PRINT(array)
    PRINT(transposed)
    PRINT(nm::shape(array))
    PRINT(nm::shape(transposed))
}


transposing a raw array
array:
[[[	1,	2,	3],
[	3,	4,	5]],

[[	5,	6,	7],
[	7,	8,	9]]]
transposed:
[[[	1,	5],
[	3,	7]],

[[	2,	6],
[	4,	8]],

[[	3,	7],
[	5,	9]]]
nm::shape(array):
[	2,	2,	3]
nm::shape(transposed):
(3,	2,	2)


### Reshaping an Array

In [12]:
auto newshape = std::array{3,2,2};

In [13]:
DESC("\nreshaping an std::array")
{
    auto array = std::array{1,2,3,4,5,6,7,8,9,10,11,12};
    auto reshaped = view::reshape(array,newshape);
    PRINT(array)
    PRINT(reshaped)
    PRINT(nm::shape(array))
    PRINT(nm::shape(reshaped))
}


reshaping an std::array
array:
[	1,	2,	3,	4,	5,	6,	7,	8,	9,	10,	11,	12]
reshaped:
[[[	1,	2],
[	3,	4]],

[[	5,	6],
[	7,	8]],

[[	9,	10],
[	11,	12]]]
nm::shape(array):
[	12]
nm::shape(reshaped):
[	3,	2,	2]


### Flattening an Array

In [14]:
DESC("\n flattening an std::array")
{
    using array_t = std::array<std::array<int,2>,2>;
    array_t array = {{{1,2},{3,4}}};
    auto flattened = view::flatten(array);
    PRINT(array)
    PRINT(flattened)
    PRINT(nm::shape(array))
    PRINT(nm::shape(flattened))
}


 flattening an std::array
array:
[[	1,	2],
[	3,	4]]
flattened:
[	1,	2,	3,	4]
nm::shape(array):
[	2,	2]
nm::shape(flattened):
(4)


### Repeating an Array

In [15]:
auto repeats = std::array{1,2};
int axis = 1;

In [16]:
DESC("\n repeating an std::array")
{
    using array_t = std::array<std::array<int,2>,2>;
    array_t array = {{{1,2},{3,4}}};
    auto repeated = view::repeat(array,repeats,axis);
    PRINT(array)
    PRINT(repeated)
    PRINT(nm::shape(array))
    PRINT(nm::shape(repeated))
}


 repeating an std::array
array:
[[	1,	2],
[	3,	4]]
repeated:
[[	1,	2,	2],
[	3,	4,	4]]
nm::shape(array):
[	2,	2]
nm::shape(repeated):
[	2,	3]


### Tiling an Array

In [17]:
auto reps = std::tuple{2,1,2};

In [18]:
DESC("\n tiling an std::array")
{
    auto array = std::array{0,1,2};
    auto tiled = view::tile(array,reps);
    PRINT(array)
    PRINT(tiled)
    PRINT(nm::shape(array))
    PRINT(nm::shape(tiled))
}


 tiling an std::array
array:
[	0,	1,	2]
tiled:
[[[	0,	1,	2,	0,	1,	2]],

[[	0,	1,	2,	0,	1,	2]]]
nm::shape(array):
[	3]
nm::shape(tiled):
[	2,	1,	6]


### Squeeze and Expand Dims

In [19]:
DESC("\nsqueezing an std::array")
{
    using array_t = std::array<std::array<int,1>,3>;
    array_t array = {1,2,3};
    auto squeezed = view::squeeze(array);
    PRINT(array)
    PRINT(squeezed)
    PRINT(nm::shape(array))
    PRINT(nm::shape(squeezed))
}


squeezing an std::array
array:
[[	1],
[	2],
[	3]]
squeezed:
[	1,	2,	3]
nm::shape(array):
[	3,	1]
nm::shape(squeezed):
(3)


In [20]:
DESC("\nexpanding an std::array")
{
    auto axis = std::array{0,2};
    auto array = std::array{1,2,3};
    auto expanded = view::expand_dims(array,axis);
    PRINT(array)
    PRINT(expanded)
    PRINT(nm::shape(array))
    PRINT(nm::shape(expanded))
}


expanding an std::array
array:
[	1,	2,	3]
expanded:
[[[	1],
[	2],
[	3]]]
nm::shape(array):
[	3]
nm::shape(expanded):
[	1,	3,	1]
